<a href="https://colab.research.google.com/github/AdriCon02/Deep-Fore/blob/main/YOLO/finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPU Check

In [ ]:
!nvidia-smi

# General settings

In [ ]:
import os
# Set YOLOv8 to quiet mode
#os.environ['YOLO_VERBOSE'] = 'False'
HOME = os.getcwd()
print(HOME)

# Install dependencies

In [ ]:
# Pip install method (recommended)

# Install and update Ultralytics and Ray Tune packages
!pip install -U ultralytics==8.0.196
!pip install grpcio==1.59.0 ray==2.8.1

# Optionally install W&B for logging
!pip install wandb

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
%pip uninstall -y albumentations

from IPython.display import display, Image

import ray
from ray import tune


# Import refined Dataset with SAM

In [ ]:
!mkdir {HOME}/datasets
%cd {HOME}/datasets

!pip install roboflow --quiet


#OBJECT DETECTION (TUMOR & MITOSIS)
from roboflow import Roboflow
rf = Roboflow(api_key="UeLjBGp1wL0ROOeH62zO")
project = rf.workspace("yolo-brecahad-5ejws").project("yolo4class")
version = project.version(5)
dataset = version.download("yolov8")


#OBJECT DETECTION (LUMEN & NO LUMEN)
# from roboflow import Roboflow
# rf = Roboflow(api_key="UeLjBGp1wL0ROOeH62zO")
# project = rf.workspace("yolo-brecahad-5ejws").project("yolo4class")
# version = project.version(4)
# dataset = version.download("yolov8")




# Fine Tuning w/ Grid Search

In [ ]:
ray.shutdown()

In [ ]:
from ultralytics import YOLO
from ray import tune

# Load a YOLOv8n model
model = YOLO("yolov8n.pt")

# # Run Ray Tune on the model
result_grid = model.tune(
    data="/content/datasets/Yolo4Class-4/data.yaml",
    space={"lr0": tune.uniform(1e-5, 1e-3),
          "lrf": tune.uniform(0.01, 1.0),
          "momentum": tune.uniform(0.6, 0.98),
          "weight_decay":tune.uniform(0.0, 0.001),
          "warmup_epochs":tune.uniform(0.0, 5.0),
          "warmup_momentum":tune.uniform(0.0, 0.95),
          "box":tune.uniform(0.02, 10),
          "cls":tune.uniform(0.2, 4.0),
          "hsv_h":tune.uniform(0.0, 0.0),
          "hsv_s":tune.uniform(0.0, 0.0),
          "hsv_v":tune.uniform(0.0, 0.0),
          "degrees":tune.uniform(0.0, 0.0),
          "translate":tune.uniform(0.0, 0.9),
          "scale":tune.uniform(0.0, 0.0),
          "shear":tune.uniform(0.0, 0.0),
          "perspective":tune.uniform(0.0, 0.0),
          "flipud":tune.uniform(0.0, 0.0),
          "fliplr":tune.uniform(0.0, 0.0),
          "mosaic":tune.uniform(0.0, 0.0),
          "mixup":tune.uniform(0.0, 0.0),
          "copy_paste":tune.uniform(0.0, 0.0)
          },
    epochs=30,
    use_ray=True,
    gpu_per_trial=1  # Specify GPU resources
 )


# Download trained model and results

In [ ]:
import shutil
import os

# # Ruta de la carpeta que deseas descargar en Google Colab
ruta_local = '/content/runs'

# # Ruta donde deseas guardar el archivo ZIP en Google Colab
archivo_zip = '/content/runs.zip'

# # Crear un archivo ZIP de la carpeta
shutil.make_archive('/content/runs', 'zip', ruta_local)

# # Descargar el archivo ZIP
from google.colab import files
files.download(archivo_zip)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>